<a href="https://colab.research.google.com/github/s00hyun/Today-I-Learned/blob/master/ML-DL/deep-learning-pytorch/lec5_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [10]:
!pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=8f38dcadfb11b43f36d53955da92cb9f7260bfad07ef686887621e967223f2d9
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [0]:
import import_ipynb

In [7]:
!pwd

/content


In [17]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/딥러닝수업

/content/gdrive/My Drive/Colab Notebooks/딥러닝수업


In [18]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/딥러닝수업


In [19]:
!ls -l lec5_model.ipynb

-rw------- 1 root root 5687 Apr 14 06:35 lec5_model.ipynb


In [0]:
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [23]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("current device:", device)

current device: cuda:0


In [0]:
num_epochs, num_classes, batch_size, learning_rate = 10, 10, 16, 0.01

In [0]:
from lec5_model import ConvNet, AlexNet, VGGNet

In [0]:
network = 'simple-cnn'  # simple-cnn, alexnet, vgg-16

In [0]:
if network == 'simple-cnn':
  model = ConvNet().to(device)
  composed_transforms = transforms.Compose([transforms.Resize((32, 32)),  # 해당 네트워크에 맞게 input size 조정
                                            # 텐서 Type으로 변환
                                            # Type: PIL -> torch.Tensor
                                            # Pixel value: [0, 255] -> [0, 1]
                                            transforms.ToTensor(),   
                                            # (R채널 Mean, G채널 Mean, B채널 Mean), (R채널 Std, G채널 Std, B채널 Std)
                                            # Pixel value를 [0, 1]에서 [-1, 1]로 변환함
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
elif network == 'alexnet':
  model = AlexNet(num_classes=num_classes).to(device)
  composed_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
elif network == 'vgg-16':
  model = VGGNet(num_classes=num_classes).to(device)
  composed_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [27]:
print(model)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
)


In [30]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=composed_transforms, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=composed_transforms)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [33]:
# 모델에 학습 데이터를 입력 -> 예측 레이블 산출, 정답 레이블과 비교 -> 모델의 파라미터 값 업데이트

for epoch in range(num_epochs):
  # 각 data batch에 대해 학습
  # (image, labels): (입력 이미지, 정답 레이블)
  for i, (images, labels) in enumerate(train_loader):
    # to(device): 연산 수행 디바이스 설정
    # images, labels, model간 연산을 위해.
    images = images.to(device)  
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()

    if (i + 1) % 1000 == 0:
      print('Epoch: {}/{}, Batch Step: {}/{}, Loss: {:.4f}, Training Accuracy of the Current Batch: {}%'.
            format(epoch + 1, num_epochs, i + 1, train_loader.__len__(), loss.item(), 100 * correct / batch_size))


Epoch: 1/10, Batch Step: 1000/3125, Loss: 2.1762, Training Accuracy of the Current Batch: 25.0%
Epoch: 1/10, Batch Step: 2000/3125, Loss: 2.1371, Training Accuracy of the Current Batch: 31.25%
Epoch: 1/10, Batch Step: 3000/3125, Loss: 1.9024, Training Accuracy of the Current Batch: 56.25%
Epoch: 2/10, Batch Step: 1000/3125, Loss: 1.9981, Training Accuracy of the Current Batch: 43.75%
Epoch: 2/10, Batch Step: 2000/3125, Loss: 2.0277, Training Accuracy of the Current Batch: 43.75%
Epoch: 2/10, Batch Step: 3000/3125, Loss: 2.1503, Training Accuracy of the Current Batch: 31.25%
Epoch: 3/10, Batch Step: 1000/3125, Loss: 2.0716, Training Accuracy of the Current Batch: 37.5%
Epoch: 3/10, Batch Step: 2000/3125, Loss: 1.9298, Training Accuracy of the Current Batch: 56.25%
Epoch: 3/10, Batch Step: 3000/3125, Loss: 2.2106, Training Accuracy of the Current Batch: 25.0%
Epoch: 4/10, Batch Step: 1000/3125, Loss: 2.1486, Training Accuracy of the Current Batch: 31.25%
Epoch: 4/10, Batch Step: 2000/312

In [34]:
model.eval()  # 모델 평가 모드로 변경 (모델 변수 고정 등)
with torch.no_grad():  # 테스트 과정에서 gradient 계산 배제
  total, correct = 0, 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy of the 10,000 Test Images: {}%'.format(100 * correct / total))

Test Accuracy of the 10,000 Test Images: 40.37%
